<a href="https://colab.research.google.com/github/jeyasubha-26/mindmapGenerator/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%writefile main.py
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

model_name = "google/flan-t5-base"   # 🔥 CHANGED
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

class MindmapRequest(BaseModel):
    text: str
    complexity: int = 3

def summarize_if_needed(text, complexity):
    if len(text.split()) > 120 and complexity < 7:
        prompt = f"Summarize the following text:\n{text}"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
        outputs = model.generate(**inputs, max_new_tokens=120)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text

def generate_mindmap_logic(text, complexity):
    processed_text = summarize_if_needed(text, complexity)

    prompt = (
        f"Create a hierarchical mindmap.\n"
        f"Complexity: {complexity}/10\n"
        f"Text:\n{processed_text}"
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=200)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    lines = result.split(". ")
    tree = "🧠 Mindmap\n"
    for i, line in enumerate(lines):
        tree += "  " * (i % (complexity + 1)) + "├─ " + line + "\n"

    return tree

@app.post("/generate-mindmap")
def generate_mindmap(req: MindmapRequest):
    return {"mindmap": generate_mindmap_logic(req.text, req.complexity)}


Overwriting main.py


In [9]:
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 &


nohup: appending output to 'nohup.out'


In [10]:
!pip install pyngrok


In [11]:
from pyngrok import ngrok

ngrok.set_auth_token("my authtoken")


In [12]:
from pyngrok import ngrok
ngrok.kill()
url = ngrok.connect(8000)
print(url)


NgrokTunnel: "https://1902fd645723.ngrok-free.app" -> "http://localhost:8000"
